In [ ]:
# Import packages
import numpy as np
import pandas as pd

In [ ]:
# Load the rental listings
rental_listings = pd.read_csv("../data/raw/rental_listings_for_capital_cities.csv")
rental_listings.head()

In [ ]:
# Load the properties for sale
listings_for_sale = pd.read_csv("../data/raw/properties_for_sale_in_capital_cities.csv")
listings_for_sale.head()